# Mise en place d'une solution locale pour traitement par un LLM 

Cette solution se base sur [ollama](https://ollama.com). Il est nécessaire d'installer l'application qui va aussi installer la ligne de commande *ollama*, puis la librairie Python du même nom via *pip*.

Dans l'exemple ci-dessous, nous allons utiliser l'un des modèles proposés : llama3.1. Il faut donc le télécharger localement sur votre machine via la commande suivante dans un terminal : *ollama pull llama3.1*

Pour l'encodage du sens des documents, il faut installer la librairie *sentence_transformers*.

In [9]:
#On importe les librairies utiles
# !pip install ollama faiss-gpu # if you use GPU
# !pip install ollama faiss-cpu # if you use CPU
import ollama
import os
import numpy as np
from tqdm.autonotebook import tqdm, trange
from sentence_transformers import SentenceTransformer
import faiss
import math

Pour travailler uniquement en local, on a besoin d'un modèle de type *Sentence Transformer* pour plonger les documents. L'un des plus performants aujourd'hui est peut-être *bget-small-en* et il en existe spécialisés sur le français.

In [3]:
model =  SentenceTransformer("ggrn/bge-small-en")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/17.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Chargement du corpus, comme d'habitude mais en imposant une taille minimum aux documents/paragraphes.

In [5]:
with open("alice.txt") as f:    
    lines = [line.strip() for line in f.readlines()]

print(f"{len(lines)} lignes dans le fichier texte")

docs = []
s = ""
for l in lines:
    if l:
        s += " " + l
    elif s:
        if len(s.split(" ")) > 5: # more than 5 words in text
            docs.append(s)
        s = ""

print(f"{len(docs)} paragraphes")

5777 lignes dans le fichier texte
916 paragraphes


In [11]:
build_embedding = True # mettre True la première fois pour calculer et sauvegarder les plongements

if build_embedding:
    
    steps = 200 # nombre de documents traités à la fois
    
    embeddings_full = np.zeros((len(docs), 384), dtype=np.float32) # 384 est la taille des plongements

    num_batches = math.floor(len(docs)/steps)
    for batch_num in tqdm(range(num_batches)):

        embeddings = model.encode(docs[batch_num*steps:(batch_num+1)*steps], show_progress_bar=True)
        embeddings_full[batch_num*steps:(batch_num+1)*steps] = embeddings
    
    embeddings = model.encode(docs[num_batches*steps:])
    embeddings_full[num_batches*steps:] = embeddings
    np.save("emb_alice", embeddings_full)
    
else:
    embeddings_full = np.load("emb_alice.npy")

  0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

In [13]:
# A noter que l'index pourrait être, lui aussi, sauvegardé pour éviter de ré-indexer à chaque fois

dim = embeddings_full.shape[1]
index = faiss.IndexFlatL2(dim)
print(dim, index.is_trained)
index.add(embeddings_full)

384 True


On construit une fonction qui prend en entrée la question posée puis réalise tout le traitement, à savoir :

- plongement de la question = vecteur encodant le sens de la question (*embedding*)
- recherche dans l'index en précisant le nombre de documents retournés (paramètre k)
- mis au point de l'invite (*prompt*) en utilisant les documents retrouvés et la question
- interrogation du LLM via *ollama*
- retourne la réponse

In [2]:
def poser_une_question(question):
    question_embedding = np.array([model.encode(question)])
    _, indices = index.search(question_embedding, k=1)
    retrieved_docs = docs[indices[0][0]]
    #print(retrieved_docs) # à décommenter si vous voulez afficher les documents retrouvés
    messages = [
            {
                "role": "user",
                "content": f"""
            Context information is below.
            ---------------------
                {retrieved_docs}
            ---------------------
            Given the context information and not prior knowledge, answer the query.
                Query: {question}
            Answer:
            """
            }
        ]
    response = ollama.chat(model='llama3.1', messages=
            messages
        )
    return response['message']['content']

In [3]:
question1 = "What unexpected animal the duchess comes out carrying in baby's clothes?"
question2 = "What is the usual fate reserved for the queen's enemies?"
question3 = "Who carries the crown on a cushion?"

print(poser_une_question(question1))

NameError: name 'np' is not defined